In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns
import os, sys
import tensorflow.compat.v1 as tf
from skimage import transform
from skimage import data

In [11]:
def load_data(data_directory):
    
    #getting the directories in each folder and then we will did the files in the directory
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    
    # looking for the pmm files only and then putting the names in directory
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) if f.endswith(".ppm")]
        
        # reading the images using matplotlib
        for f in file_names:
            images.append(plt.imread(f))
            labels.append(int(d))
    return images, labels

In [12]:
train_data_directory = "data/BelgiumTSC_Training/Training"
images, labels = load_data(train_data_directory)

In [14]:
type(images)

list

In [15]:
images_array = np.array(images)
labels_array = np.array(labels)

In [16]:
print(images_array.ndim)

1
